# D10 Biến đổi cấu trúc (reshaping)

## Mục đích

Giới thiệu các cách thay đổi cấu trúc số liệu trong Pandas.


## Stacking và unstacking

Cách làm này thường áp dụng cho các index nhiều cấp. Chúng ta sẽ quan sát lại ví dụ trong bài trước.

In [1]:
import pandas as pd

d = pd.read_excel("../assets/hrm.xlsx", index_col="id")
d_agg = d.groupby("sex")[["height", "weight"]].agg(["mean", "std"])
d_agg

height               weight          
         mean       std       mean       std
sex                                         
0    1.557678  0.044879  52.045662  5.920553
1    1.670381  0.059999  63.285714  8.388080

Nếu đã có một bảng tổng kết số liệu như thế này và bạn lại muốn tạo ra một cột "Mean (SD)" từ bảng này, cách đơn giản nhất là như sau. Chúng ta sẽ xem từng bước và mình sẽ giải thích sau mỗi bước.

In [2]:
d_agg_stacked = d_agg.stack(0)
d_agg_stacked

mean       std
sex                            
0   height   1.557678  0.044879
    weight  52.045662  5.920553
1   height   1.670381  0.059999
    weight  63.285714  8.388080

Chúng ta vừa "stack" số liệu trong bảng. Stacking là chuyển một hoặc nhiều cấp của **cột** xuống **index**. Trong lệnh trên, mình stack cấp có số thứ tự `0`, tương ứng với tên các biến (`"height"`, `"weight"`). Số thứ tự của cột được tính từ trên xuống dưới.

Lúc này bạn có thể dùng hàm `apply()`.

In [3]:
d_agg_stacked_apply = d_agg_stacked.apply(lambda x: "{:.2f} ({:.2f})".format(*x), axis=1)
d_agg_stacked_apply

sex        
0    height     1.56 (0.04)
     weight    52.05 (5.92)
1    height     1.67 (0.06)
     weight    63.29 (8.39)
dtype: object

Rất có thể chúng ta sẽ muốn so sánh chiều cao và cân nặng giữa hai giới tính. Thông thường, chúng ta sẽ đặt hai giới tính ở hai cột song song với nhau cho dễ so sánh. Chúng ta sẽ "unstack" theo giới tính. Ngược lại với stacking, unstacking chuyern một hoặc nhiều cấp của **index** sang **cột**.

In [4]:
d_agg_stacked_apply.unstack("sex")

sex,0,1
height,1.56 (0.04),1.67 (0.06)
weight,52.05 (5.92),63.29 (8.39)


Trong ví dụ vừa xong, cấp "giới tính" trong index của data frame `d_agg_stacked_apply` có tên là `"sex"`. Với một cấp có tên, bạn có thể sử dụng tên của cấp thay cho số thứ tự của nó.

Chúng ta có thể "chain" các dòng lệnh trên như sau.

In [5]:
d.groupby("sex")[["height", "weight"]].agg(["mean", "std"]) \
    .stack(0) \
    .apply(lambda x: "{:.2f} ({:.2f})".format(*x), axis=1) \
    .unstack("sex")

sex,0,1
height,1.56 (0.04),1.67 (0.06)
weight,52.05 (5.92),63.29 (8.39)


Trong trường hợp không cung cấp đối số về cấp sẽ được stack hoặc unstack, Pandas mặc định stack / unstack cấp xa nhất.

In [6]:
d_agg.stack()

height     weight
sex                          
0   mean  1.557678  52.045662
    std   0.044879   5.920553
1   mean  1.670381  63.285714
    std   0.059999   8.388080

In [7]:
d_agg_stacked_apply.unstack()

,height,weight
sex,,
0,1.56 (0.04),52.05 (5.92)
1,1.67 (0.06),63.29 (8.39)


## Pivoting

Bên cạnh công cụ stacking và unstacking rất lợi hại, bạn có thể tái cấu trúc nhanh một data frame bằng `pivot()`. Pivoting cho phép bạn xác định cột nào sẽ được dùng làm tên index, cột nào dùng làm tên cột, và cột nào chứa các giá trị tương ứng với index và cột. Hãy cùng xem ví dụ sau.

In [8]:
d2 = pd.DataFrame({
    "city": ["A", "B", "A", "B"],
    "season": [1, 1, 2, 2],
    "temp": [30, 21, 25, 18]
})
d2

,city,season,temp
0,A,1,30
1,B,1,21
2,A,2,25
3,B,2,18


Pivoting sẽ phù hợp nếu bạn muốn biểu diễn bảng này dưới dạng các hàng là thành phố, các cột là mùa, và số liệu trong bảng là nhiệt độ tương ứng với mỗi thành phố trong mỗi mùa.

In [9]:
d2.pivot(index="city", columns="season", values="temp")

season,1,2
city,,
A,30,25
B,21,18


Pivoting sẽ tự động thêm các giá trị NA vào những dòng và cột không có giá trị tương ứng.

In [10]:
d3 = pd.concat(
    [
        d2,
        pd.DataFrame({"city": ["A"], "season": [3], "temp": [17]})
    ],
    ignore_index=True
)
d3

,city,season,temp
0,A,1,30
1,B,1,21
2,A,2,25
3,B,2,18
4,A,3,17


In [11]:
d3_pivot = d3.pivot(index="city", columns="season", values="temp")
d3_pivot

season,1,2,3
city,,,
A,30.0,25.0,17.0
B,21.0,18.0,NaN


## Dữ liệu dạng wide và long

Trong ví dụ trên, chúng ta thấy hai dạng của dữ liệu đo lường lặp lại (repeated measurements). Ở dạng dữ liệu "wide", mỗi hàng là một cá thể / đối tượng đo, còn kết quả của các lần đo được ghi nhận trong các cột khác nhau (bảng trên). Ở dạng dữ liệu "long", mỗi cá thể có nhiều hàng, mỗi hàng là một lần đo.

In [12]:
d3_wide = d3_pivot.reset_index().rename_axis(columns=None)
d3_wide

,city,1,2,3
0,A,30.0,25.0,17.0
1,B,21.0,18.0,NaN


Dạng dữ liệu "long" phù hợp để thu thập và lưu trữ số liệu, vì trong một lần đo bạn có thể đo lường nhiều nội dung khác nhau, tuy nhiên, trong một số phân tích thì dạng dữ liệu "wide" sẽ phù hợp hơn. Do vậy, chúng ta cần biết cách chuyển qua lại giữa các dạng dữ liệu này.

Việc chuyển dữ liệu từ long sang wide được tiến hành thông qua pivoting (đã được giới thiệu ở trên). Còn việc chuyển dữ liệu từ wide sang long được tiến hành thông qua melting. Trong khi các công cụ giới thiệu trên đưa các cột vào thành tên index và tên cột, melting tạo ra các cột mới trong data frame.

In [13]:
d3_wide.melt(id_vars="city", var_name="season", value_name="temp")

,city,season,temp
0,A,1,30.0
1,B,1,21.0
2,A,2,25.0
3,B,2,18.0
4,A,3,17.0
5,B,3,NaN


Bạn có thể thấy rằng bằng hàm `melt()`, chúng ta "tái tạo" được data frame `d3`. Hàm này có vài đối số:

* `id_vars`: danh sách các cột sẽ được giữ lại, tất cả các cột còn lại trong data frame sẽ được "melt".
* `var_name`: tên của cột chứa tên của các cột được "melt".
* `value_name`: tên của cột chứa các giá trị nằm trong các cột được "melt".

Bạn có thể thấy rằng khác với data frame `d3` ban đầu, data frame này có thêm một dòng NA của thành phố B, mùa 3. Chúng ta có thể loại bỏ các dòng này bằng hàm `dropna()` đã học.

In [14]:
d3_wide.melt(id_vars="city", var_name="season", value_name="temp").dropna(subset="temp")

,city,season,temp
0,A,1,30.0
1,B,1,21.0
2,A,2,25.0
3,B,2,18.0
4,A,3,17.0


## Tùy biến cách tổng hợp số liệu với `groupby()` và các hàm reshaping

Ở phần đầu chúng ta thống kê trung bình và độ lệch chuẩn như sau.

In [15]:
d.groupby("sex")[["height", "weight"]].agg(["mean", "std"]).stack(0)

mean       std
sex                            
0   height   1.557678  0.044879
    weight  52.045662  5.920553
1   height   1.670381  0.059999
    weight  63.285714  8.388080

Bạn cũng có thể "stack" số liệu của các cột trước rồi sau đó mới nhóm và tổng hợp số liệu.

In [16]:
d[["sex", "height", "weight"]].set_index("sex").stack()

sex        
1    height     1.69
     weight    65.00
     height     1.67
     weight    65.00
     height     1.70
               ...  
0    weight    40.00
     height     1.54
     weight    49.00
     height     1.63
     weight    60.00
Length: 640, dtype: float64

In [17]:
d[["sex", "height", "weight"]].set_index("sex").stack().groupby(level=[0, 1]).agg(["mean", "std"])

mean       std
sex                            
0   height   1.557678  0.044879
    weight  52.045662  5.920553
1   height   1.670381  0.059999
    weight  63.285714  8.388080

Trong ví dụ trên, chúng ta đã chuyển hết các cột vào index, do đó, khi `groupby()`, chúng ta không sử dụng tên cột mà sẽ cung cấp các cấp mà chúng ta muốn nhóm lại. Đây là một cách nhóm dữ liệu khác mà mình muốn giới thiệu với các bạn.

Bạn cũng có thể sử dụng hàm `melt()`.

In [18]:
d[["sex", "height", "weight"]].melt(id_vars="sex")

,sex,variable,value
0,1,height,1.69
1,1,height,1.67
2,1,height,1.70
3,1,height,1.71
4,0,height,1.45
...,...,...,...
655,0,weight,47.00
656,0,weight,58.00
657,0,weight,40.00
658,0,weight,49.00


In [19]:
d[["sex", "height", "weight"]].melt(id_vars="sex") \
    .groupby(["sex", "variable"])["value"] \
    .agg(["mean", "std"])

mean       std
sex variable                     
0   height     1.557678  0.044879
    weight    52.045662  5.920553
1   height     1.670381  0.059999
    weight    63.285714  8.388080

Bạn có thể gói các lệnh này vào trong một hàm để đỡ phải gõ lại nhiều lần khi phân tích số liệu.

In [20]:
def calc_mean_sd(d: pd.DataFrame, vars: list, groupby: list):
    return d[groupby + vars].melt(id_vars=groupby) \
        .groupby(groupby + ["variable"])["value"] \
        .agg(["mean", "std"])

vars = ["height", "weight"]
groupby = ["sex"]
calc_mean_sd(d, vars, groupby)

mean       std
sex variable                     
0   height     1.557678  0.044879
    weight    52.045662  5.920553
1   height     1.670381  0.059999
    weight    63.285714  8.388080

---

[Bài trước](./09_transform.ipynb) - [Danh sách bài](../README.md) - [Bài sau](./11_tabulation.ipynb)